In [ ]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory, Preprocessor
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory, Preprocessor
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

def run_preprocessor_checks():
    methods = ["data quality", "fairness"]
    ohe_flags = [True] #[False, True]

    for index, data in enumerate(DATASETS.items()):
        data_link = data[0]
        info = data[1]
        if data_link in ["https://www.kaggle.com/datasets/aemreusta/brfss-2020-survey-data",
                        "https://www.kaggle.com/datasets/fleshmetal/records-a-comprehensive-music-metadata-dataset",
                        "https://www.kaggle.com/datasets/sudipde25/lifespan-calorie-burn-dynamics"]:
            continue
        target_column = info["target_column"]
        nr_file = info["no_dataset"]
        # 1) CSV laden
        ingestor_factory = IngestorFactory(data_link, nr_file, False)
        ingestor = ingestor_factory.create()
        df = ingestor.load_data()
        print(f"\n==> Testing {data_link!r}  target={target_column}")

        for method in methods:
            for ohe in ohe_flags:
                label = f"  [{method:12s} | ohe={ohe}]"
                try:
                    # 2) Factory + Preprocessor
                    factory = PreprocessorFactory(df, method, target_column)
                    pre     = factory.create()

                    # 3) Aufruf process_data
                    result = pre.process_data(ohe=ohe)

                    # 4) Struktur prüfen
                    if not isinstance(result, tuple):
                        raise AssertionError(f"{label} ➞ returned no tuple")
                    if len(result) != 6:
                        raise AssertionError(f"{label} ➞ expected 6 items, got {len(result)}")

                    df_tr, num_cols, cat_cols, text_feats, sens_cols, tgt_col = result

                    # 5a) DataFrame-Typ und Zeilenzahl
                    if not isinstance(df_tr, pd.DataFrame):
                        raise AssertionError(f"{label} ➞ first item not a DataFrame")
                    if df_tr.shape[0] != df.shape[0]:
                        raise AssertionError(f"{label} ➞ row count changed {df_tr.shape[0]} vs {df.shape[0]}")

                    # 5b) Target-Spalte vorhanden
                    if target_column not in df_tr.columns:
                        raise AssertionError(f"{label} ➞ target column '{target_column}' missing")
                    
                    # Nan Check
                    if df_tr.isnull().values.any():
                        nan_cols = df_tr.columns[df_tr.isnull().any()].tolist()
                        raise AssertionError(f"{label} ➞ found NaNs in columns {nan_cols}")

                    # 6) Die vier Listen müssen Listen sein
                    for name, lst in [
                        ("numeric_columns",    num_cols),
                        ("categorical_columns",cat_cols),
                        ("text_features",      text_feats),
                        ("sensitive_columns",  sens_cols)
                    ]:
                        if not isinstance(lst, list):
                            raise AssertionError(f"{label} ➞ '{name}' is not a list")

                    # 7) target_column-Return
                    if not isinstance(tgt_col, str) or tgt_col != target_column:
                        raise AssertionError(f"{label} ➞ returned target '{tgt_col}'") 
                    
                    # Every thing is numerical
                    if ohe:
                        for col in df_tr.columns:
                            if not pd.api.types.is_numeric_dtype(df_tr[col]):
                                raise AssertionError(
                                    f"{label} ➞ column '{col}' is not numeric (dtype={df_tr[col].dtype})"
                                )
                    
                    
                    vals = df_tr[target_column].unique()
                    counts = df_tr[target_column].value_counts().to_dict()
                    if len(vals) != 2:
                        raise AssertionError(
                            f"{label} ➞ target column must be binary, "
                            f"but found values {vals}"
                        )
                    else:
                        print(f"{label} → target value counts: {counts}")

                    print(f"{label} ✓")
                except Exception as e:
                    print(f"{label} ✗ ERROR: {e}")

    print("\nAll checks done.")


run_preprocessor_checks()

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Creating CSV from Arrow format: 100%|██████████| 33/33 [00:00<00:00, 388.24ba/s]



==> Testing 'https://huggingface.co/datasets/scikit-learn/adult-census-income'  target=income
  [data quality | ohe=True] → target value counts: {1: 24720, 0: 7841}
  [data quality | ohe=True] ✓
  [fairness     | ohe=True] → target value counts: {1: 24720, 0: 7841}
  [fairness     | ohe=True] ✓
student_habits_performance.csv
Dataset URL: https://www.kaggle.com/datasets/jayaantanaath/student-habits-vs-academic-performance
../../data/kaggle_temp/student_habits_performance.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/jayaantanaath/student-habits-vs-academic-performance'  target=exam_score
  [data quality | ohe=True] → target value counts: {0: 502, 1: 498}
  [data quality | ohe=True] ✓
  [fairness     | ohe=True] → target value counts: {0: 502, 1: 498}
  [fairness     | ohe=True] ✓
student_info.csv
Dataset URL: https://www.kaggle.com/datasets/therohithanand/student-academic-performance-dataset
../../data/kaggle_temp/student_info.csv was deleted.

==> Testing 'https://www

Creating CSV from Arrow format: 100%|██████████| 100/100 [00:00<00:00, 500.71ba/s]



==> Testing 'https://huggingface.co/datasets/marianeft/diabetes_prediction_dataset'  target=diabetes
  [data quality | ohe=True] → target value counts: {1: 91500, 0: 8500}
  [data quality | ohe=True] ✓
  [fairness     | ohe=True] → target value counts: {1: 91500, 0: 8500}
  [fairness     | ohe=True] ✓


Creating CSV from Arrow format: 100%|██████████| 30/30 [00:00<00:00, 169.49ba/s]


...

tested.csv
Dataset URL: https://www.kaggle.com/datasets/brendan45774/test-file
../../data/kaggle_temp/tested.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/brendan45774/test-file'  target=Survived
  [data quality | ohe=True] → target value counts: {1: 266, 0: 152}
  [data quality | ohe=True] ✓
  [fairness     | ohe=True] → target value counts: {1: 266, 0: 152}
  [fairness     | ohe=True] ✓
speeddating.csv
Dataset URL: https://www.kaggle.com/datasets/ulrikthygepedersen/speed-dating
../../data/kaggle_temp/speeddating.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/ulrikthygepedersen/speed-dating/data'  target=match
  [data quality | ohe=True] → target value counts: {1: 6998, 0: 1380}
  [data quality | ohe=True] ✓
  [fairness     | ohe=True] → target value counts: {1: 6998, 0: 1380}
  [fairness     | ohe=True] ✓
451
<class 'str'>

==> Testing 'https://www.openml.org/search?type=data&sort=runs&status=active&id=451'  target=Leaving_Certificate
  [data quality 

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\sklearn\preprocessing\_discretization.py:334: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn(


fertility.csv
Dataset URL: https://www.kaggle.com/datasets/muhamedyoussry/fertility-data-set
../../data/kaggle_temp/fertility.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/muhamedyoussry/fertility-data-set'  target=Diagnosis
  [data quality | ohe=True] → target value counts: {1: 88, 0: 12}
  [data quality | ohe=True] ✓
  [fairness     | ohe=True] → target value counts: {1: 88, 0: 12}
  [fairness     | ohe=True] ✓
physics_teacher_jobs_linkedin.csv
Dataset URL: https://www.kaggle.com/datasets/kushalmanage/physics-teacher-jobs-linkedin
../../data/kaggle_temp/physics_teacher_jobs_linkedin.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/kushalmanage/physics-teacher-jobs-linkedin'  target=contractType
  [data quality | ohe=True] → target value counts: {1: 711, 0: 136}
  [data quality | ohe=True] ✓
  [fairness     | ohe=True] → target value counts: {1: 711, 0: 136}
  [fairness     | ohe=True] ✓
Public Vs Private Universties.csv
Dataset URL: https://www.kaggle.c

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\src\ingestion\ingestorFactory.py:380: DtypeWarning: Columns (20,50,161,200) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(final_csv)



==> Testing 'https://www.kaggle.com/datasets/threnjen/portland-housing-prices-sales-jul-2020-jul-2021'  target=price
  [data quality | ohe=True] → target value counts: {0: 12883, 1: 12848}
  [data quality | ohe=True] ✓
  [fairness     | ohe=True] → target value counts: {0: 12883, 1: 12848}
  [fairness     | ohe=True] ✓
duplicate_deleted.csv
Dataset URL: https://www.kaggle.com/datasets/janakpariyar/coursera-courses-uncleaned-dataset-to-practice
../../data/kaggle_temp/duplicate_deleted.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/janakpariyar/coursera-courses-uncleaned-dataset-to-practice'  target=course_rating
  [data quality | ohe=True] → target value counts: {0: 32130, 1: 18037}
  [data quality | ohe=True] ✓
  [fairness     | ohe=True] → target value counts: {0: 32130, 1: 18037}
  [fairness     | ohe=True] ✓

All checks done.


In [3]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory, Preprocessor
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

data_link = "https://www.kaggle.com/datasets/danofer/law-school-admissions-bar-passage"
nr_file = 1
target_column = "bar_passed"
ingestor_factory = IngestorFactory(data_link, nr_file, False)
ingestor = ingestor_factory.create()
data = ingestor.load_data()
dqp = Preprocessor(data, target_column="")
numeric_columns, categorical_columns = dqp.receive_categorized_columns()
preprocessor_factory = PreprocessorFactory(data, "fairness", target_column=target_column)
preprocessor = preprocessor_factory.create()
processed_data_dict = preprocessor.process_data(ohe=True)


processed_data_dict[0]

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bar_pass_prediction.csv
Dataset URL: https://www.kaggle.com/datasets/danofer/law-school-admissions-bar-passage
../../data/kaggle_temp/bar_pass_prediction.csv was deleted.


c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\sklearn\preprocessing\_discretization.py:334: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn(


,sex_1.0,sex_2.0,race_1.0,race_2.0,race_3.0,race_4.0,race_5.0,race_6.0,race_7.0,race_8.0,age_0-9,gender_female,gender_male,race1_asian,race1_black,race1_hisp,race1_other,race1_white,race2_a black,race2_b white,race2_c other,Dropout_NO,Dropout_YES,asian_0,asian_1,bar_a Passed 1st time,bar_b Passed 2nd time,bar_c Failed,bar_e non-Grad,bar1_F,bar1_P,bar1_yr_1.0,bar1_yr_2.0,bar1_yr_6.0,bar1_yr_7.0,bar1_yr_<MISSING>,bar2_F,bar2_P,bar2_yr_93.0,bar2_yr_94.0,bar2_yr_95.0,bar2_yr_96.0,bar2_yr_<MISSING>,black_0,black_1,cluster_1.0,cluster_2.0,cluster_3.0,cluster_4.0,cluster_5.0,cluster_6.0,cluster_<MISSING>,decile1_1.0,decile1_10.0,decile1_2.0,decile1_3.0,decile1_4.0,decile1_5.0,decile1_6.0,decile1_7.0,decile1_8.0,decile1_9.0,decile1_<MISSING>,decile1b_1.0,decile1b_10.0,decile1b_2.0,decile1b_3.0,decile1b_4.0,decile1b_5.0,decile1b_6.0,decile1b_7.0,decile1b_8.0,decile1b_9.0,decile1b_<MISSING>,decile3_1.0,decile3_10.0,decile3_2.0,decile3_3.0,decile3_4.0,decile3_5.0,decile3_6.0,decile3_7.0,decile3_8.0,decile3_9.0,decile3_<MISSING>,fam_inc_1.0,fam_inc_2.0,fam_inc_3.0,fam_inc_4.0,fam_inc_5.0,fam_inc_<MISSING>,fulltime_1.0,fulltime_2.0,fulltime_<MISSING>,grad_<MISSING>,grad_O,grad_X,grad_Y,hisp_0,hisp_1,indxgrp_a under 400,indxgrp_b 400-460,indxgrp_c 460-520,indxgrp_d 520-580,indxgrp_e 580-640,indxgrp_f 640-700,indxgrp_g 700+,indxgrp2_a under 400,indxgrp2_b 400-460,indxgrp2_c 460-520,indxgrp2_d 520-580,indxgrp2_e 580-640,indxgrp2_f 640-700,indxgrp2_g 700-760,indxgrp2_h 760-820,indxgrp2_i 820+,male_0.0,male_1.0,male_<MISSING>,other_0,other_1,parttime_0.0,parttime_1.0,parttime_<MISSING>,pass_bar_0,pass_bar_1,tier_1.0,tier_2.0,tier_3.0,tier_4.0,tier_5.0,tier_6.0,tier_<MISSING>,ID,lsat,ugpa,zfygpa,DOB_yr,zgpa,index6040,dnn_bar_pass_prediction,gpa,bar_passed
0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,-1.701914,1.335336,0.707808,1.389893,0.713466,1.996640,1.392582,-0.077679,0.707808,1
1,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,-1.701914,-1.441997,0.707808,-0.219838,0.713466,-0.617230,-0.924012,-0.077679,0.707808,1
2,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,-1.701914,-0.145908,0.707808,-0.812308,-0.101378,-1.204018,0.157065,-0.077679,0.707808,1
3,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,-1.701914,0.409558,0.707808,0.283203,-1.527355,-0.531879,0.620384,-0.077679,0.707808,1
4,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,-1.701914,2.075958,0.707808,1.043353,0.509755,0.801728,2.010340,-0.077679,0.707808,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [3]:
target_column in numeric_columns

False

In [4]:
target_column in categorical_columns

True

In [5]:
"pass_bar" in categorical_columns

True

In [13]:
processed_data_dict[2]

['Dropout_NO',
 'Dropout_YES',
 'asian_0',
 'asian_1',
 'bar_a Passed 1st time',
 'bar_b Passed 2nd time',
 'bar_c Failed',
 'bar_e non-Grad',
 'bar1_F',
 'bar1_P',
 'bar1_yr_1.0',
 'bar1_yr_2.0',
 'bar1_yr_6.0',
 'bar1_yr_7.0',
 'bar1_yr_<MISSING>',
 'bar2_F',
 'bar2_P',
 'bar2_yr_93.0',
 'bar2_yr_94.0',
 'bar2_yr_95.0',
 'bar2_yr_96.0',
 'bar2_yr_<MISSING>',
 'black_0',
 'black_1',
 'cluster_1.0',
 'cluster_2.0',
 'cluster_3.0',
 'cluster_4.0',
 'cluster_5.0',
 'cluster_6.0',
 'cluster_<MISSING>',
 'decile1_1.0',
 'decile1_10.0',
 'decile1_2.0',
 'decile1_3.0',
 'decile1_4.0',
 'decile1_5.0',
 'decile1_6.0',
 'decile1_7.0',
 'decile1_8.0',
 'decile1_9.0',
 'decile1_<MISSING>',
 'decile1b_1.0',
 'decile1b_10.0',
 'decile1b_2.0',
 'decile1b_3.0',
 'decile1b_4.0',
 'decile1b_5.0',
 'decile1b_6.0',
 'decile1b_7.0',
 'decile1b_8.0',
 'decile1b_9.0',
 'decile1b_<MISSING>',
 'decile3_1.0',
 'decile3_10.0',
 'decile3_2.0',
 'decile3_3.0',
 'decile3_4.0',
 'decile3_5.0',
 'decile3_6.0',
 'dec

In [5]:
data

,Êcommunityname,state,countyCode,communityCode,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
0,BerkeleyHeightstownship,NJ,39,5320,1,11980,3.10,1.37,91.78,6.50,...,14,114.85,138,1132.08,16,131.26,2,16.41,41.02,1394.59
1,Marpletownship,PA,45,47616,1,23123,2.82,0.80,95.57,3.44,...,57,242.37,376,1598.78,26,110.55,1,4.25,127.56,1955.95
2,Tigardcity,OR,<MISSING>,,1,29344,2.43,0.74,94.33,3.43,...,274,758.14,1797,4972.19,136,376.3,22,60.87,218.59,6167.51
3,Gloversvillecity,NY,35,29443,1,16656,2.40,1.70,97.35,0.50,...,225,1301.78,716,4142.56,47,271.93,<MISSING>,,306.64,
4,Bemidjicity,MN,7,5068,1,11245,2.76,0.53,89.16,1.17,...,91,728.93,1060,8490.87,91,728.93,5,40.05,NaN,9988.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210,Mercedcity,CA,<MISSING>,,10,56216,3.07,6.87,61.68,15.23,...,1376,2268.72,2563,4225.82,489,806.25,34,56.06,545.75,7356.84
2211,Pinevillecity,LA,<MISSING>,,10,12251,2.68,21.18,76.65,1.52,...,104,860.43,574,4748.9,24,198.56,2,16.55,124.1,5824.44
2212,Yucaipacity,CA,<MISSING>,,10,32824,2.46,0.52,92.62,0.98,...,628,1709.26,895,2435.97,179,487.19,8,21.77,353.83,4654.2
2213,Beevillecity,TX,<MISSING>,,10,13547,2.89,3.37,69.91,0.90,...,192,1508.01,474,3722.9,13,102.1,1,7.85,691.17,5340.87


In [2]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

data_link = "https://www.kaggle.com/datasets/ehtishamsadiq/uncleaned-laptop-price-dataset"
nr_file = 1
target_column = "Price"
ingestor_factory = IngestorFactory(data_link, nr_file, False)
ingestor = ingestor_factory.create()
data = ingestor.load_data()
preprocessor_factory = PreprocessorFactory(data, "fairness", target_column=target_column)
preprocessor = preprocessor_factory.create()
processed_data_dict = preprocessor.process_data(ohe=True)
processed_data_dict[0]

laptopData.csv
Dataset URL: https://www.kaggle.com/datasets/ehtishamsadiq/uncleaned-laptop-price-dataset
../../data/kaggle_temp/laptopData.csv was deleted.


,Company_<MISSING>,Company_Acer,Company_Apple,Company_Asus,Company_Chuwi,Company_Dell,Company_Fujitsu,Company_Google,Company_HP,Company_Huawei,...,TypeName_<MISSING>,TypeName_Gaming,TypeName_Netbook,TypeName_Notebook,TypeName_Ultrabook,TypeName_Workstation,Unnamed: 0,Weight_svd_0,Weight_svd_1,Price
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,-1.720688,3.667499e-07,-4.386292e-07,1
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,-1.720688,2.090653e-09,-3.991545e-08,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,-1.720688,2.539585e-07,3.409556e-05,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,-1.720688,3.729272e-10,2.291879e-08,1
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,-1.720688,3.667499e-07,-4.386292e-07,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.711762,-6.945907e-08,-1.051442e-05,0
1299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.711762,-4.740568e-09,9.999999e-01,1
1300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1.711762,2.128474e-08,7.631897e-06,0
1301,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1.711762,-9.723566e-10,-2.320377e-07,0


In [1]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

data_link = "https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset"
nr_file = 1
target_column = "stroke"
ingestor_factory = IngestorFactory(data_link, nr_file, False)
ingestor = ingestor_factory.create()
data = ingestor.load_data()
preprocessor_factory = PreprocessorFactory(data, "fairness", target_column=target_column)
preprocessor = preprocessor_factory.create()
processed_data_dict = preprocessor.process_data(ohe=True)
processed_data_dict[0]

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


healthcare-dataset-stroke-data.csv
Dataset URL: https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset
../../data/kaggle_temp/healthcare-dataset-stroke-data.csv was deleted.


,gender_Female,gender_Male,gender_Other,age_0-9,age_10-19,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,...,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,id,avg_glucose_level,bmi,stroke
0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,-1.299046,2.729251,1.051577,0
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0.716749,2.139974,-0.095996,0
2,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,-0.255637,-0.002831,0.498042,0
3,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,1,1.118962,1.450556,0.754558,0
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,-1.648063,1.514869,-0.649531,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,-0.864584,-0.496194,-0.095996,1
5106,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0.395064,0.426220,1.510606,1
5107,1,0,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,-0.794175,-0.513107,0.241525,1
5108,0,1,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0.048506,1.340623,-0.433518,1


In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)